In [1]:
from generate_declarative_sentences import generate_declarative_statements
from categorise_sentences import QuestionCategoryAssigner, StatementCategoryAssigner
from dataset_processing import DatasetPreProcessor
from config.global_config import GlobalConfig
from training import AnnabellPreTrainingRunner

global_config = GlobalConfig()

model_name = "qwen3:4b"
if 'declarative_sentences_dataset' not in locals():
    import pandas as pd

    declarative_sentences_dataset = pd.read_json('declarative_sentences_dataset_sample.jsonl', lines=True)
else:
    #generate a small sample of declarative sentences from the NYC SQuAD dataset
    declarative_sentences_dataset = generate_declarative_statements(5, model_name)
    #use this dataset to assign categories to the questions and statements
    question_assigner = QuestionCategoryAssigner(declarative_sentences_dataset)
    question_assigner.generate_statement_categories(model_name)
    statement_assigner = StatementCategoryAssigner(declarative_sentences_dataset)
    statement_assigner.generate_statement_categories(model_name)
    declarative_sentences_dataset.to_json("declarative_sentences_dataset_sample.jsonl", orient="records", lines=True)
#apply the formatting rules to the dataset
datasetPreProcessor = DatasetPreProcessor(declarative_sentences_dataset)
datasetPreProcessor.preprocess_data()
#select the pretraining data
datasetPreProcessor.select_pretraining_data(global_config.percentage_of_pre_training_samples())
datasetPreProcessor.create_commands_for_pretraining()
declarative_sentences_dataset.to_json(global_config.prepared_dataset_with_commands_filepath(), orient="records",
                                      lines=True)
print(f"dataset saved to file: {global_config.prepared_dataset_with_commands_filepath()}")
declarative_sentences_dataset

2025-11-14 18:40:39,532 - root - INFO - Logging initialized. Log file: /Users/chris/logs/cognitive_language_model_logs/run_20251114_184039.log
2025-11-14 18:40:40,997 - dataset_processing - INFO - Number of pretraining samples: 2
2025-11-14 18:40:40,998 - dataset_processing - INFO - Samples per category: 1
2025-11-14 18:40:41,000 - dataset_processing - INFO - Pretraining samples by question category:
2025-11-14 18:40:41,001 - dataset_processing - INFO - question_category
Subject-Verb-Object    1
Name: count, dtype: int64
2025-11-14 18:40:41,001 - dataset_processing - INFO - Pretraining samples by sentence category:
2025-11-14 18:40:41,002 - dataset_processing - INFO - statement_category
Subject-Verb-Object    1
Name: count, dtype: int64
2025-11-14 18:40:41,002 - dataset_processing - INFO - Total number of samples selected for pretraining: 1
dataset saved to file: /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/

,id,title,context,question,declarative_statement,question_category,statement_category,answer,declarative_statement_formatted,question_formatted,answer_formatted,is_pretraining,created_commands
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,the Virgin Mary allegedly appeared to Saint Be...,Subject-Verb-Object,Subject-Verb-Object,Saint Bernadette Soubirous,the_Virgin_Mary allegedly appear to Saint_Bern...,? to whom do the_Virgin_Mary allegedly appear ...,Saint_Bernadette_Soubirous,False,[# This is a non-pretraining sample. No comman...
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ is in front of the N...,Subject-Verb-Object,Subject-Verb-Object,a copper statue of Christ,a copper statue of Christ be in front of the_N...,? what be in front of the_Notre_Dame Main Buil...,a copper statue of Christ,True,"[#id: 5733be284776f4190066117f, Subject-Verb-O..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,The Basilica of the Sacred Heart at Notre Dame...,Subject-Verb-Object,Subject-Verb-Object,the Main Building,the Basilica of the Sacred Heart at Notre_Dame...,? the Basilica of the sacred heart at Notre_Da...,the_Main_Building,False,[# This is a non-pretraining sample. No comman...
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,The Grotto at Notre Dame is a Marian place of ...,Subject-Verb-Object,Subject-Verb-Object,a Marian place of prayer and reflection,the Grotto at Notre_Dame be a marian place of ...,? what be the Grotto at Notre_Dame,a marian place of prayer and reflection,False,[# This is a non-pretraining sample. No comman...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary sits on top...,Subject-Verb-Object,Subject-Verb-Object,a golden statue of the Virgin Mary,a golden statue of the_Virgin_Mary sit on top ...,? what sit on top of the_Main_Building at Notr...,a golden statue of the_Virgin_Mary,False,[# This is a non-pretraining sample. No comman...


In [3]:
#todo add POS tagging to columns
#fix pretraining selection algorithm
# use the POS tagging to identify goals as part of command generation

In [3]:
#run the pretraining using the generated dataset

from dataset_processing import DatasetPreProcessor
from config.global_config import GlobalConfig
from training import AnnabellPreTrainingRunner
import pandas as pd

global_config = GlobalConfig()

ds_filepath = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/data/nyc_squad_with_pretraining_commands.jsonl"

declarative_sentences_dataset = pd.read_json(ds_filepath, lines=True)
datasetPreProcessor = DatasetPreProcessor(declarative_sentences_dataset)

runner = AnnabellPreTrainingRunner(datasetPreProcessor)
runner.run()

2025-11-15 07:44:24,176 - dataset_processing - INFO - Wrote /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/sandbox/pre_training/nyc_squad_pretraining_commands.txt
2025-11-15 07:44:24,176 - dataset_processing - INFO - Number of reward lines: 1
2025-11-15 07:44:24,177 - dataset_processing - INFO - Number of commands: 13
2025-11-15 07:44:24,178 - training - INFO - copied: /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/sandbox/pre_training/nyc_squad_pretraining_commands.txt to: /Users/chris/PycharmProjects/Training-and-evaluating-cognitive-language-models/docker/shared_data/pre_training
2025-11-15 07:44:24,178 - dataset_processing - INFO - Wrote /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/ann

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/logging/__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/logging/__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/logging/__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/logging/__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/chris/PycharmProjects/Trai